# Accessing ProPublica API and Converting Form 990 Information to Pandas Dataframes #

In [1]:
import requests
import pandas as pd
import numpy as np
from tqdm import tqdm

#### The function below will access the ProPublica API and return all nonprofit entities with the provided query string in the organization title and the National Taxonomy of Exempt Entities (NTEE) code. More information on NTEE codes is available here:#### 
https://learn.guidestar.org/help/ntee-codes 

#### Documentation of the ProPublica API is available here: ####
https://projects.propublica.org/nonprofits/api

In [5]:
def ProPublica_API_access (query_string, ntee_code):
    """
    This function retrieves organizations from the ProPublica API matching a given query string, i.e. 'opera', 'symphony', etc.
    
    Args:
    query_string (str): A query string that must be a part of the name of each organization that is returned.
    ntee_code (str): The National Taxonomy of Exempt Entities (NTEE) code
    
    Returns:
    A Pandas dataframe 
    """
    
    url = 'https://projects.propublica.org/nonprofits/api/v2/search.json?q='+query_string
    organizations = []
    for x in range(11):
        params = dict(q=query_string, page=x)
        request = requests.get(url, params=params)
        json = request.json()
        json.pop('total_results')
        page_of_orgs = json.pop('organizations')
        organizations.append(page_of_orgs)
    global df
    df = pd.DataFrame()
    for x in tqdm(range(len(organizations))):
        for y in range(len(organizations[x])):
            org = organizations[x][y]
            df = df.append(org,ignore_index=True)
    #This step will fix typos in the 'ntee_code' column#
    df['ntee_code'].fillna(ntee_code,inplace=True)
    df[df['ntee_code'].str.contains(ntee_code)]
    #This step sets the organization's EIN as the index#
    df['ein'] = df['ein'].astype('int')
    df = df.set_index('ein')
    return df.head()

In [16]:
ProPublica_API_access('opera','A6A')

100%|██████████████████████████████████████████████████████████████████████████████████| 11/11 [00:08<00:00,  1.34it/s]


,city,has_subseccd,have_extracts,have_filings,have_pdfs,name,ntee_code,raw_ntee_code,score,state,strein,sub_name,subseccd
ein,,,,,,,,,,,,,
813311094,ASTORIA,1.0,None,None,None,BARE OPERA,A6A,A6A,8022.336,NY,81-3311094,BARE OPERA,3.0
263016871,LEXINGTON,1.0,None,None,None,BLUEGRASS OPERA,A6A,A6A,8022.336,KY,26-3016871,BLUEGRASS OPERA,3.0
833139772,SUNLAND,1.0,None,None,None,MISSION OPERA,A6A,A6A,8022.336,CA,83-3139772,MISSION OPERA,3.0
462541609,BEND,1.0,None,None,None,OPERA BEND,A99,A99,8022.336,Oregon,46-2541609,OPERA BEND,3.0
840850527,ENGLEWOOD,1.0,None,None,None,OPERA COLORADO,A6A0,A6A0,8022.336,CO,84-0850527,OPERA COLORADO,3.0


In [18]:
df['strein'] = df['strein'].str.replace('-','')
df['strein'] = pd.to_numeric(df['strein'], errors = 'coerce')

AttributeError: Can only use .str accessor with string values!

In [27]:
eins = df.index.values.tolist()
eins

[813311094,
 263016871,
 833139772,
 462541609,
 840850527,
 464332937,
 453361602,
 222519789,
 800432320,
 824079263,
 237314812,
 850391704,
 821536854,
 824602315,
 910760426,
 752253647,
 364454720,
 463486730,
 814363604,
 260006536,
 953854105,
 566019660,
 432032363,
 231504706,
 133819493,
 522175213,
 20484003,
 462223000,
 930679959,
 475311222,
 471255941,
 237207572,
 454913985,
 203975320,
 10496860,
 541618192,
 952224201,
 943068290,
 726021455,
 200156714,
 824935291,
 952503791,
 954512214,
 463142361,
 50453691,
 510213334,
 571129329,
 411375724,
 680454224,
 263649993,
 756004746,
 522001426,
 471571156,
 721584393,
 222270100,
 270940187,
 920148120,
 237169261,
 832435880,
 954868027,
 262584694,
 341816013,
 811599946,
 770668548,
 463775780,
 223954352,
 475324014,
 942495701,
 990197758,
 263346265,
 261577184,
 461846033,
 833135996,
 260348842,
 821447688,
 270813566,
 382052726,
 272295236,
 621119830,
 886009267,
 942985556,
 203520577,
 300540629,
 133098

In [75]:
url2 = 'https://projects.propublica.org/nonprofits/api/v2/organizations/{}.json'.format(840850527)
request2 = requests.get(url2)
json2 = request2.json()
json2

{'organization': {'id': 840850527,
  'ein': 840850527,
  'name': 'OPERA COLORADO',
  'careofname': None,
  'address': '4121 S NAVAJO ST STE 100',
  'city': 'ENGLEWOOD',
  'state': 'CO',
  'zipcode': '80110-4479',
  'exemption_number': 0,
  'subsection_code': 3,
  'affiliation_code': 3,
  'classification_codes': '1200',
  'ruling_date': '1981-11-01',
  'deductibility_code': 1,
  'foundation_code': 15,
  'activity_codes': '090000000',
  'organization_code': 1,
  'exempt_organization_status_code': 1,
  'tax_period': '2019-06-01',
  'asset_code': 6,
  'income_code': 7,
  'filing_requirement_code': 1,
  'pf_filing_requirement_code': 0,
  'accounting_period': 6,
  'asset_amount': 2438450,
  'income_amount': 5176971,
  'revenue_amount': 5047106,
  'ntee_code': 'A6A0',
  'sort_name': None,
  'created_at': '2020-06-16T00:21:58.641Z',
  'updated_at': '2020-06-16T00:21:58.641Z',
  'data_source': None,
  'have_extracts': None,
  'have_pdfs': None},
 'filings_with_data': [{'tax_prd': 201806,
   'ta

In [65]:
import time
company_ein=[]
tax_period=[]
tot_assets=[]
tot_income=[]
cont_income=[]
earned_income=[]
tot_revenue=[]
pct_compnsatncurrofcr=[]
tax_year = []
totassetsend=[]
totliabend=[]
totfuncexpns=[]
invest_income=[]
admin_expenses=[]
yearly_revenue=[]
other_revenue=[]
skipped_requests=[]
for ein in tqdm(eins):
    try:
        url2 = 'https://projects.propublica.org/nonprofits/api/v2/organizations/{}.json'.format(ein)
        request2 = requests.get(url2)
        json2 = request2.json()
        if len(json2['filings_with_data']) == 0:
            company_ein.append(ein)
            tax_period.append(json2['organization']['tax_period'])
            tot_assets.append(json2['organization']['asset_amount'])
            tot_income.append(json2['organization']['income_amount'])
            tot_revenue.append(json2['organization']['revenue_amount'])
            tax_year.append(np.nan)
            admin_expenses.append(np.nan)
            cont_income.append(np.nan)
            earned_income.append(np.nan)
            invest_income.append(np.nan)
            pct_compnsatncurrofcr.append(np.nan)
            totassetsend.append(np.nan)
            totliabend.append(np.nan)
            totfuncexpns.append(np.nan)
            yearly_revenue.append(np.nan)
            other_revenue.append(np.nan)
            time.sleep(0.5)
        else:
            for filing in range(len(json2['filings_with_data'])):
                company_ein.append(ein)
                tax_period.append(json2['organization']['tax_period'])
                tot_assets.append(json2['organization']['asset_amount'])
                tot_income.append(json2['organization']['income_amount'])
                tot_revenue.append(json2['organization']['revenue_amount'])
                tax_year.append(json2['filings_with_data'][filing]['tax_prd_yr'])
                admin_expenses.append(json2['filings_with_data'][filing]['compnsatncurrofcr'])
                cont_income.append(json2['filings_with_data'][filing]['totcntrbgfts'])
                earned_income.append(json2['filings_with_data'][filing]['totprgmrevnue'])
                invest_income.append(json2['filings_with_data'][filing]['invstmntinc'])
                pct_compnsatncurrofcr.append(json2['filings_with_data'][filing]['pct_compnsatncurrofcr'])
                totassetsend.append(json2['filings_with_data'][filing]['totassetsend'])
                totliabend.append(json2['filings_with_data'][filing]['totliabend'])
                totfuncexpns.append(json2['filings_with_data'][filing]['totfuncexpns'])
                yearly_revenue.append(json2['filings_with_data'][filing]['totrevenue'])
                other_revenue.append(json2['filings_with_data'][filing]['miscrevtot11e'])
            time.sleep(0.5)
    except:
        tax_period.append(json2['organization']['tax_period'])
        tot_assets.append(json2['organization']['asset_amount'])
        tot_income.append(json2['organization']['income_amount'])
        tot_revenue.append(json2['organization']['revenue_amount'])
        tax_year.append(np.nan)
        admin_expenses.append(np.nan)
        cont_income.append(np.nan)
        earned_income.append(np.nan)
        invest_income.append(np.nan)
        pct_compnsatncurrofcr.append(np.nan)
        totassetsend.append(np.nan)
        totliabend.append(np.nan)
        totfuncexpns.append(np.nan)
        yearly_revenue.append(np.nan)
        other_revenue.append(np.nan)

100%|████████████████████████████████████████████████████████████████████████████████| 917/917 [20:16<00:00,  1.33s/it]


In [66]:
df2 = pd.DataFrame(
    list(zip(company_ein, tax_period, tot_assets, tot_income, tot_revenue, pct_compnsatncurrofcr, tax_year, cont_income, earned_income, invest_income, other_revenue, yearly_revenue, totassetsend, totliabend, totfuncexpns, admin_expenses)),
    columns = ['ein','tax_period', 'total_assets', 'total_income', 'total_revenue', 'pct_exp__current_officers', 'tax_year', 'contributed_income','earned_income','investment_income','other_income','year_end_revenue','year_end_assets', 'year_end_liabilities', 'total_functional_expenses','admin_expenses'])
df2.head(10)

,ein,tax_period,total_assets,total_income,total_revenue,pct_exp__current_officers,tax_year,contributed_income,earned_income,investment_income,other_income,year_end_revenue,year_end_assets,year_end_liabilities,total_functional_expenses,admin_expenses
0,813311094,2019-12-01,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,263016871,2018-12-01,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,833139772,2019-12-01,1280.0,26521.0,26521.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,462541609,2018-12-01,12908.0,53745.0,50416.0,NaN,2017.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,840850527,2018-12-01,12908.0,53745.0,50416.0,0.0,NaN,2792062.0,1385097.0,91.0,7033.0,4123180.0,2069154.0,2048030.0,4426594.0,290481.0


In [67]:
df2.shape

(2428, 16)

In [68]:
print(len(skipped_requests))

0


In [70]:
df2.to_csv('Form 990s.csv')

In [71]:
df.to_csv('Non-profit entities.csv')